## Simulations on structural distortions induced by electric field

### Theory

The force on each atom

$$F_{\kappa, \alpha}(\mathit{R_{\kappa},E})=\sum_{\beta} Z^*_{\kappa,\alpha\beta} E_{\beta} - \sum_{\kappa', \alpha'\beta} C_{\alpha\alpha'}(\kappa\kappa')\tau_{\kappa', \alpha'\beta} = 0$$

$Z^*_{\kappa,\alpha\beta}$ -- Born effective charge, $E_{\beta}$ -- electric field, $C_{\alpha\alpha'}(\kappa\kappa')$ -- force constant, $\tau$ -- atomic displacement, $\alpha$, $\beta$ -- atomic position; $\kappa$ -- index of atom. 

\begin{align*}
Z^*_{\kappa,\alpha,\beta} E_{\beta} &= \sum_{\kappa', \alpha'} C_{\alpha\alpha'}(\kappa\kappa') \tau_{\kappa', \alpha'\beta}\\
&= \sum_{m, \kappa', \alpha'} C_{\alpha\alpha'}(\kappa\kappa') \tau_{m, \beta} \eta_m (\kappa', \alpha') \\
&= \sum_{m} \omega_m^2 M_{\kappa} \tau_{m, \beta} \eta_{m}(\kappa, \alpha)
\end{align*}

$$\sum_{\kappa, \alpha} Z^*_{\kappa,\alpha,\beta} \eta_m (\kappa, \alpha) E_{\beta} =  \sum_{\kappa, \alpha}\eta_{m}(\kappa, \alpha) M_{\kappa} \eta_{m}(\kappa, \alpha) \tau_{m, \beta} \omega_m^2$$

$\eta_{m}(\kappa, \alpha)$ -- eigen-displacementsince. Since $\sum_{\kappa, \alpha}\eta_{m}(\kappa, \alpha) M_{\kappa} \eta_{m}(\kappa, \alpha) = M_m = 1$,

$$\tau_{m} = \frac{1}{\omega_m^2 M_m}\sum_{\kappa, \alpha} Z^*_{\kappa,\alpha,\beta} \eta_{m}(\kappa, \alpha) \cdot E_{\beta} $$

$$\tau_{\kappa, \beta} = \sum_{m, \alpha}\frac{1}{\omega_m^2 M_m} Z^*_{\kappa,\alpha,\beta} \eta_{m}(\kappa, \alpha) \cdot E_{\beta} $$

### Units

\begin{align*}
[\omega_m] &= (\frac{eV}{\unicode{x212B} * AMU})^{1/2}\\
[M_m] &= AMU \\
[Z^*] &= e \\
[E_{\beta}] &= V/\unicode{x212B} \\
[\tau] &= \unicode{x212B}
\end{align*}

<p>AMU -- atomic mass unit</p>

In [2]:
# From Phonopy github https://github.com/atztogo/phonopy/blob/master/phonopy/units.py

# AMU = 1.6605402e-27 # [kg]
# Joule = 1.0         # [kg m^2 / s^2]
# EV = 1.60217733e-19 # [J]
# Angstrom = 1.0e-10  # [m]
# THz = 1.0e12        # [s^-1]
# PI = 3.1415926

# ConvTHz = (EV/AMU)**0.5/Angstrom/(2*PI)/1e12 # [THz] 15.633302
# print ConvTHz**2
# kV/cm = 10^{-5} V/Angstrom

### Codes

This Code is for calculation on the Electric field - displacement constants.

Inputs:
- Phonon modes at Gamma point
- Born effective charges

Input file -- OUTCAR from VASP

In [4]:
#----------------------------------------------------------------------
# Import operator
#----------------------------------------------------------------------
import numpy as np
from numpy import array as npa
import math

# Set print precision
np.set_printoptions(precision=4, suppress=True)

#----------------------------------------------------------------------
# Define functions to transform myList to float/int
#----------------------------------------------------------------------
def myFloat(myList):
    return map(float, myList)
def myInt(myList):
    return map(int, myList)

#----------------------------------------------------------------------
# Constants for unit conversion
#----------------------------------------------------------------------
EV = 1.60217733e-19                         # eV to J
Angstrom = 1.0e-10                          # A (Angstron) to m
AMU = 1.6605402e-27                         # amu (atom mass unit) to kg
PI = 3.1415926                              # Pi

#----------------------------------------------------------------------
# Define a find_str() function -- record the index of line containing 
# the string you are looking for.
# The string should be the only one in the text whole
#----------------------------------------------------------------------
def find_str(str):
    line = 0
    for ln in whole:
        line = line + 1
        if str in ln:
            return(line-1)
            break

#----------------------------------------------------------------------
# Read the whole file
#----------------------------------------------------------------------
out = open ('OUTCAR', 'r')
whole = [line.split() for line in out]

#----------------------------------------------------------------------
# Read all phonon informations 
#----------------------------------------------------------------------
ln = find_str("NIONS")
le = find_str("Eigenvectors")               # locate 

nions = int(whole[ln][-1])                  # number of ions in the unit cell
nf = 3*nions                                # number of modes

eigen = whole[le+4:le+nf*(nions+3)+3]       # the whole phonon part

#----------------------------------------------------------------------
# Read eigenvalues 
#----------------------------------------------------------------------
f = []
for i in range(0, nf*(nions+3), nions+3):
    for j in range(len(eigen[i])):
        if eigen[i][j] == "THz" and float(eigen[i][j-1])>0.1:
            f.append(float(eigen[i][j-1]))
nf = len(f)
#print f, nf

#----------------------------------------------------------------------
# Read mass of each atom, Mk
#----------------------------------------------------------------------
m = [j for j, ln in enumerate(whole) if 'POMASS' in ln]
ms = myFloat(whole[m[-1]][2:])
tp = [j for j, ln in enumerate(whole) if 'ions' in ln and 'per' in ln]
type = myInt(whole[tp[0]][4:])
Mk = []
for i in range(len(ms)):
    Mk = np.append(Mk, np.full((1,type[i]),ms[i]))
#print Mk

#----------------------------------------------------------------------
# Read eigenvectors gamma (output from VASP)
#----------------------------------------------------------------------
gamma = []
for m in range(nf):
    for j in range(m*23+2,m*23+22):
        gamma = gamma + myFloat(eigen[j][3:6])
gamma = np.vsplit(npa(gamma).reshape(-1,3), nf)
#print gamma

#----------------------------------------------------------------------
# Get eigendisplacement eta
#----------------------------------------------------------------------
eta = []
for m in range(nf):
    for i in range(nions):
        eta = np.append(eta, gamma[m][i]/Mk[i]**0.5)
eta = np.vsplit(npa(eta).reshape(-1,3), nf)
#print eta[0]

#----------------------------------------------------------------------
# Read Born Effective Charges
#----------------------------------------------------------------------
lb = find_str("cummulative")
whole_bec = whole[lb+2:lb+4*nions+2]
bec = []
for i in range(nions):
    bec = bec+whole_bec[4*i+1:4*i+4]
BEC =  np.vsplit(np.delete(npa(map(myFloat, bec)), np.s_[0], axis=1), nions)
#for i in range(nions):
#print BEC[i]

#----------------------------------------------------------------------
# Calc factor from unit 
# Unit of Electric field is kV/cm
# Unit of eigenvalues from VASP is THz
#----------------------------------------------------------------------
ConvTHz =(EV/Angstrom**2/AMU)**0.5/(2*PI)/1e12
fu = ConvTHz**2*10**(-5)

#print fu*10**5
#----------------------------------------------------------------------
# Get displacement for each atom, unit Angst
#----------------------------------------------------------------------
R = []
for i in range(nions):
    zy = npa([0.0,0.0,0.0])
    for m in range(nf):
        zeff = np.dot(BEC[i], eta[m][i])/f[m]**2
        zy = np.add(zy, fu*zeff)
    R = np.hstack((R, zy))
dp = R.reshape(-1,3)

#print dp
#np.savetxt('cons.txt', dp, fmt='%12.9f')

E = 500 # kV/cm
print "Displacement along x by applying electric field along x:\n", E*dp.T[0]
print "Displacement along y by applying electric field along y:\n", E*dp.T[1]
print "Displacement along z by applying electric field along z:\n", E*dp.T[2]

# End, but it needs to be improved...

Displacement along x by applying electric field along x:
[ 0.0388 -0.0114 -0.0334 -0.0124  0.0239  0.0502 -0.0351  0.0014  0.0031
  0.0098  0.002  -0.018  -0.0128 -0.0206 -0.0168 -0.0303  0.0124  0.0124
  0.0039  0.0156]
Displacement along y by applying electric field along y:
[ 0.0291  0.0072 -0.042   0.0159 -0.0187 -0.0086  0.022  -0.0123 -0.0073
  0.0063  0.0069 -0.0054 -0.0031  0.0152  0.0156 -0.0153  0.0003 -0.0039
 -0.0049  0.0079]
Displacement along z by applying electric field along z:
[ 0.0522  0.0082 -0.0149 -0.0209 -0.0036 -0.0034 -0.0105 -0.014   0.0069
  0.0018  0.0128  0.0041  0.0053 -0.0021  0.0042 -0.0013  0.0068  0.0048
  0.0007  0.0408]


In [24]:
## Check the normalization of VASP output
for m in range(nf):
    tot = 0
    for i in range(nions):
        tot = tot + np.dot(gamma[m][i], gamma[m][i])
#    print tot

In [ ]:
#----------------------------------------------------------------------
# calc mode polarity p = Z* eta
#----------------------------------------------------------------------
p = []
for m in range(nf):
    s = npa([0.0,0.0,0.0])
    for i in range(nions):
        s = np.add(s, np.dot(BEC[i], eta[m][i]))
    p = np.append(p, s)
p = p.reshape(-1, 3)
#print p

#----------------------------------------------------------------------
# calc mode polarity p = Z* gamma
#----------------------------------------------------------------------
ps = []
for m in range(nf):
	s = npa([0.0,0.0,0.0])
	for i in range(nions):
		s = np.add(s, np.dot(BEC[i], gamma[m][i]))
	ps = np.append(ps, s)
ps = ps.reshape(-1, 3)
#print ps


"""
zeff = []
for i in range(nions):
	z = npa([0.0,0.0,0.0])
	for m in range(nf):
		z = np.add(z, p[m][i])
	zeff = np.hstack((zeff, z))
Z = zeff.reshape(-1,3)
#print Z
"""

"""
# Get displacement for each mode, unit Angst
R = []
for m in range(nf):
	zy = npa([0.0,0.0,0.0])
	for i in range(nions):
		zeff = np.dot(BEC[i], eta[m][i])/f[m]**2
		zy = np.add(zy, fu*zeff)
	R = np.hstack((R, zy))
dp = R.reshape(-1,3)
"""

"""
# Print all
for m in range(nf):
	G = 0.0
	Y = 0.0
	print "Mode ", m+1
	print "Frequcence = ", f[m], "THz"
	print "Eigenvector		Eigendisplacement"
	for i in range(nions):
		print gamma[m][i], eta[m][i]
		G = G + np.dot(gamma[m][i],gamma[m][i])
		Y = Y + np.dot(eta[m][i],eta[m][i])*Mk[i]
	print "Polarity(eta):", p[m]
	print "Polarity(gamma):", ps[m]
    print "<g|g> = ", G
	print "<y|M|y> = ", Y
	print "\n"
"""

"""
# Count the polar mode
x = 0;y = 0; z = 0; nonp = 0;
X = []; Y = []; Z = []; NP = [];
for m in range(nf):
	if p[m][0]>0.0001 and p[m][1]<0.0001 and p[m][2]<0.0001:
		x += 1
		X = np.append(X, m+1)
	elif p[m][0]<0.0001 and p[m][1]>0.0001 and p[m][2]<0.0001:
		y += 1
		Y = np.append(Y, m+1)
	elif p[m][0]<0.0001 and p[m][1]<0.0001 and p[m][2]>0.0001:
		z += 1
		Z = np.append(Z, m+1)
	elif p[m][0]<0.0001 and p[m][1]<0.0001 and p[m][2]<0.0001:
		nonp += 1
		NP = np.append(NP, m+1)
print x,X
print y,Y
print z,Z
print nonp,NP
print x+y+z+nonp
"""